##Installation of the libraries

In [36]:
!pip install -q transformers

In [37]:
!pip install biopython

In [38]:
#!pip3 uninstall --yes torch torchaudio torchvision torchtext torchdata
!pip3 install torch

Torch optimization.

##All libraries needed for training

In [202]:
import os
import math
import numpy as np
import random
import logging

# Bring in PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
# Most of the examples have typing on the signatures for readability
from typing import Optional, Callable, List, Tuple
from Bio import SeqIO
# For data loading
from torch.utils.data import Dataset, IterableDataset, TensorDataset, DataLoader
import json
import glob
import gzip
import bz2

# For progress and timing
from tqdm import tqdm
import time
import shutil
from Bio.PDB import PDBList
from Bio.PDB.MMCIFParser import MMCIFParser
import re

In [203]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##Data processing

Getting rid of the sequences which are outside of the threshold (64-128).

Getting the sequence of a given file in the target folder (contains only the files with desired sequences).

In [235]:
file_path = "AF-A0A1D8PD42-F1-model_v4.cif"
file_model = "AF-A0A1D8PD42-F1-model_v4"
pdbl = PDBList()
pdbl.retrieve_pdb_file(file_path, file_format='mmCif', pdir=".")
# import the needed class
# instantiate the class to prepare the parser
cif_parser = MMCIFParser()
#structure = cif_parser.get_structure("3goe", "3goe.cif")
structure = cif_parser.get_structure(file_model, file_path)
model0 = structure[0]
chain_A = model0['A']  # and we get chain A
# dictionary converting 3-letter codes to 1-letter codes
# this is a very common need in bioinformatics of proteins
d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
 'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
 'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
 'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

sequence = []
for residue in chain_A:
    # for simplicity we can use X for heteroatoms (ions and water)
    sequence.append(d3to1.get(residue.get_resname(), 'X'))  #converts water and ions to X
print(''.join(sequence))

Desired structure doesn't exists
MSSSNTDNQYPKYINDTTPPTITLKEYDNASWASTTCLDHNPIKNQYIVVVMENPNQIVAIIDQQDNMILDILFKNAHDAHSKQEYSTK


Calculating the angles for the given sequence

In [236]:
#phi and psi
from Bio.PDB import PICIO, PDBIO
from Bio import PDB
from typing import TypedDict, Dict, Tuple
structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths

chain:PDB.Chain.Chain = list(structure.get_chains())[0]#iterator of chains, turns it into list, [0] first chain

ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord #this access the internal chain coords of the chain object

d: Dict[Tuple[PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey],
        PDB.internal_coords.Dihedron] = ic_chain.dihedra

cnt = 1
phi_angles = {}
phi_angles_list = []
psi_angles = {}
psi_angles_list = []

for key in d:
    if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':
        phi_angles[key] = d[key].angle
        phi_angles_list.append(d[key].angle)
    elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':
        psi_angles[key] = d[key].angle
        psi_angles_list.append(d[key].angle)


structure.internal_to_atom_coordinates(verbose = True)
io = PDBIO() #this is to write a pdb file again
io.set_structure(structure)#set structure, the structure you wan tin the pdb file

Putting angles in a matrix.

In [237]:
phi_angles_list.append(0)
psi_angles_list.append(0)

phi = np.asarray(phi_angles_list,dtype=np.float32)*(np.pi/180)
psi = np.asarray(psi_angles_list,dtype=np.float32)*(np.pi/180)
angles = np.vstack((psi,phi))

Changing sequence for to be used in the Prot-Bert embedding.

In [238]:
seq_example =  ' '.join(sequence)
seq_example

'M S S S N T D N Q Y P K Y I N D T T P P T I T L K E Y D N A S W A S T T C L D H N P I K N Q Y I V V V M E N P N Q I V A I I D Q Q D N M I L D I L F K N A H D A H S K Q E Y S T K'

##Embedding space creation (using Prot-Bert)

In [239]:
from transformers import BertModel, BertTokenizer
import re
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
seq = re.sub(r"[UZOB]", "X", seq_example)
encoded_input = tokenizer(seq, return_tensors='pt')

Getting the dimensions of the embedding space.

In [240]:
N , D = output.last_hidden_state.size()[1], output.last_hidden_state.size()[2]
embedding_prot_bert = output.last_hidden_state


In [241]:
print(N)
print(D)

91
1024


##Single-head self unmasked attention layer

In [242]:
class SelfAttention(nn.Module):
    def __init__(self, embed_dim: int):
        super(SelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.w_q = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_k = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_v = nn.Parameter(torch.randn(embed_dim, embed_dim))

    def forward(self, embeddings_prot_bert: torch.Tensor) -> torch.Tensor:
        Q = torch.matmul(embeddings_prot_bert, self.w_q)
        K = torch.matmul(embeddings_prot_bert, self.w_k)
        V = torch.matmul(embeddings_prot_bert, self.w_v)

        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(K.size(-1))
        attn = torch.softmax(scores, dim=-1)
        attention_output = torch.matmul(attn, V)

        return attention_output

##Encoder with attention and 2 layer FFNN

In [250]:
class TransformerModel(nn.Module):
    def __init__(self, embed_dim: int, feed_forward_dim1: int, feed_forward_dim2: int, output_dim: int = 2):
        super(TransformerModel, self).__init__()
        self.self_attention = SelfAttention(embed_dim)
        self.layer_norm1 = nn.LayerNorm(embed_dim)
        self.layer_norm2 = nn.LayerNorm(output_dim)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_dim, feed_forward_dim1),
            nn.ReLU(),
            nn.Linear(feed_forward_dim1, feed_forward_dim2),
            nn.ReLU(),
            nn.Linear(feed_forward_dim2, output_dim)
        )

    def forward(self, embeddings: torch.Tensor) -> torch.Tensor:
        attention_output = self.self_attention(embeddings)
        normalized_attention_output = self.layer_norm1(attention_output)
        ff_output = self.feed_forward(normalized_attention_output)
        #output = self.layer_norm2(ff_output)
        return ff_output


In [251]:
class TransformerTrainer:
    def __init__(self, model: nn.Module, criterion: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: np.ndarray):
        self.model = model
        self.criterion = criterion
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = torch.from_numpy(angles.T)
        self.optimizer = optim.SGD(model.parameters(), lr=0.01)
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.1)

    def train(self):
        loss_list = []
        for epoch in range(self.num_epochs):
            predictions = self.model(self.sequence)[:, :len(self.angles_tensor)]
            loss = self.criterion(predictions, self.angles_tensor)
            loss.backward(retain_graph=True)
            self.optimizer.step()
            self.optimizer.zero_grad()
            self.scheduler.step()
            loss_list.append(loss.item())

            print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {loss.item()}")



In [252]:
bert_model = BertModel.from_pretrained("Rostlab/prot_bert")
output = bert_model(**encoded_input)
embedded_pb = output.last_hidden_state
N, D = embedded_pb.size()[1], embedded_pb.size()[2]

# Initialize and train transformer model
feed_forward_dim1 = 512
feed_forward_dim2 = 256
num_epochs = 50

model = TransformerModel(embed_dim=D, feed_forward_dim1=feed_forward_dim1, feed_forward_dim2= feed_forward_dim2)
criterion = nn.MSELoss()
trainer = TransformerTrainer(model, criterion, num_epochs, embedded_pb, angles)
trainer.train()

/Users/goudarzimandanagmail.com/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([89, 2])) that is different to the input size (torch.Size([1, 89, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/50, Loss: 5.6320648193359375
Epoch 2/50, Loss: 4.891167640686035
Epoch 3/50, Loss: 4.6339921951293945
Epoch 4/50, Loss: 4.5579729080200195
Epoch 5/50, Loss: 4.545563220977783
Epoch 6/50, Loss: 4.544011116027832
Epoch 7/50, Loss: 4.543639659881592
Epoch 8/50, Loss: 4.543246269226074
Epoch 9/50, Loss: 4.542971611022949
Epoch 10/50, Loss: 4.542747497558594
Epoch 11/50, Loss: 4.542480945587158
Epoch 12/50, Loss: 4.542454242706299
Epoch 13/50, Loss: 4.5424299240112305
Epoch 14/50, Loss: 4.5424041748046875
Epoch 15/50, Loss: 4.542380332946777
Epoch 16/50, Loss: 4.542356967926025
Epoch 17/50, Loss: 4.542334079742432
Epoch 18/50, Loss: 4.542311191558838
Epoch 19/50, Loss: 4.542290687561035
Epoch 20/50, Loss: 4.542269706726074
Epoch 21/50, Loss: 4.5422468185424805
Epoch 22/50, Loss: 4.542245388031006
Epoch 23/50, Loss: 4.542243003845215
Epoch 24/50, Loss: 4.542241096496582
Epoch 25/50, Loss: 4.542239189147949
Epoch 26/50, Loss: 4.542237281799316
Epoch 27/50, Loss: 4.542234897613525
Epoc